In [1]:
import pandas as pd
import numpy as np
import wrangle
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from pydataset import data
import seaborn as sns
import matplotlib.pyplot as plt

Do your work for this exercise in a jupyter notebook named feature_engineering within the regression-exercises repo. Add, commit, and push your work.

1.  Load the tips dataset.

In [2]:
df = data('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


    a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [3]:
df['tip_percentage'] = df.tip/df.total_bill

In [4]:
df.head(3)

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587


    b. Create a column named price_per_person. This should be the total bill divided by the party size.

In [5]:
df['price_per_person'] = df.total_bill/df['size']

In [6]:
df.head(3)

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.003333


    c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

In [7]:
# predicting tip amount - total bill
# predicting tip percentage - total bill

    d. Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [8]:
X_train = df[['total_bill', 'size', 'tip_percentage', 'price_per_person']]

In [9]:
y_train = df[['tip']]

In [10]:
# K Best

f_selector = SelectKBest(score_func=f_regression, k=2)
f_selector.fit(X_train, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x7f89eac7b670>)

In [11]:
#. These are the top two are 'total_bill' and 'size'
mask = f_selector.get_support()
X_train.columns[mask]

Index(['total_bill', 'size'], dtype='object')

In [12]:
X_train_kbest = f_selector.transform(X_train)

model = LinearRegression().fit(X_train_kbest, y_train)
# ...

In [13]:
# Recursive Feature

lm = LinearRegression()
rfe = RFE(estimator=lm, n_features_to_select=2)
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [14]:
# Top two are 'total_bill' and 'tip_percentage'.
X_train.columns[rfe.support_]

Index(['total_bill', 'tip_percentage'], dtype='object')

    e. Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [15]:
X_train2 = df[['total_bill', 'size', 'tip', 'price_per_person']]

In [16]:
y_train2 = df[['tip_percentage']]

In [17]:
# K Best

f_selector = SelectKBest(score_func=f_regression, k=2)
f_selector.fit(X_train2, y_train2)

SelectKBest(k=2, score_func=<function f_regression at 0x7f89eac7b670>)

In [18]:
#. These are the top two are 'total_bill' and 'tip' 

X_train2.columns[f_selector.get_support()]

Index(['total_bill', 'tip'], dtype='object')

In [19]:
# Recursive Feature

lm = LinearRegression()
rfe = RFE(estimator=lm, n_features_to_select=2)
rfe.fit(X_train2, y_train2)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [20]:
# Top two are 'size' and 'tip'.
X_train2.columns[rfe.support_]

Index(['size', 'tip'], dtype='object')

In [21]:
pd.Series(dict(zip(X_train2.columns, rfe.ranking_))).sort_values()

size                1
tip                 1
price_per_person    2
total_bill          3
dtype: int64

    f. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

2.  Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [22]:
def select_kbest(X, y, k):
    f_selector = SelectKBest(f_regression, k)
    f_selector.fit(X, y)

    k_features = X.columns[f_selector.get_support()]
    
    return k_features

In [23]:
select_kbest(X_train, y_train, 2)

Index(['total_bill', 'size'], dtype='object')

3.  Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [24]:
def rfe(X, y, n):
    lm = LinearRegression()
    rfe = RFE(lm, n)
    rfe.fit(X, y)
    n_features = X.columns[rfe.support_]
    
    return n_features

In [25]:
rfe(X_train, y_train, 2)

Index(['total_bill', 'tip_percentage'], dtype='object')

In [26]:
# dictionary of results
     #.    pd.Series(dict(zip(X_train2.columns, rfe.ranking_))).sort_values()

4.  Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [27]:
swiss = data('swiss')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [32]:
# train, test, split

train, validate, test = wrangle.split_continuous(swiss)

In [33]:
train.shape, validate.shape, test.shape

((25, 6), (12, 6), (10, 6))

In [34]:
train.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Rolle,60.5,60.8,16,10,7.72,16.3
Lavaux,65.1,73.0,19,9,2.84,20.0
Nyone,56.6,50.9,22,12,15.14,16.7
Conthey,75.5,85.9,3,2,99.71,15.1
Yverdon,65.4,49.5,15,8,6.10,22.5


In [38]:
# Assisgn you X_train, y_train
X_train = train.drop(columns = ['Fertility'])
y_train = train['Fertility']

In [42]:
# K_Best
select_kbest(X_train, y_train, 3)

Index(['Examination', 'Catholic', 'Infant.Mortality'], dtype='object')

In [43]:
#RFE
rfe(X_train, y_train, 3)

Index(['Agriculture', 'Examination', 'Infant.Mortality'], dtype='object')